Notebook 1: PDF Pre-processing
---
In the series, we will be going from a PDF to Podcast using all open models.

The first step in getting to the podcast is finding a script, right now our logic is:

* Use any PDF on any topic
* Prompt meta-llama/Llama-3.1-8B-Instruct model to process it into a text file
* Re-write this into a podcast transcript in next notebook.

In this notebook, we will upload a PDF and save it into a .txt file using the PyPDF2 library, later we will process chunks from the text file using our featherlight model.

Step 0: Installing some required libraries
---
* Pydpdf2 --> pip install PyPDF2
* tkinter :
    * On linux : 
        sudo apt-get update
        
        sudo apt-get install python3-tk
    * On MacOs: brew install python-tk
    * On Windows: It should be there by default on the installed python version
* rich ipywidgets --> pip install ipywidgets
* transformers --> pip install transformers
* bitsandbytes --> pip install bitsandbytes
* trl --> pip install trl
* accelerate --> pip install accelerate
* torch --> You can install it depending on your system from the following link: https://pytorch.org/get-started/locally/

Step 1: Import Required Libraries
---

* Purpose: Import necessary libraries for PDF handling and creating the user interface.
* Libraries:
    * tkinter: Provides tools for creating a graphical user interface.  
    * filedialog: Part of tkinter, allows users to open file selection dialogs.
    * Optional from typing: Used for type hinting, indicating some functions might return None.    
    * PyPDF2: Key library for reading and extracting text and metadata from PDF files.
* Overall: Ensures all required libraries are available before any other code runs.

In [1]:
# Block 1: Import required libraries

# Import necessary libraries for GUI and PDF handling
import tkinter as tk  # GUI framework for the file dialog
from tkinter import filedialog  # To allow users to browse and select files
from typing import Optional  # Type hinting for optional return values
import PyPDF2  # Library for reading and extracting data from PDF files

Step 2: Initialize PDF Paths List and Define File Selection Function
---

* Purpose: Initialize a list to store PDF paths and create a function that lets users select files.
* Function: select_pdfs()
    * Opens a file dialog, allowing users to select multiple PDF files.
    * Stores the paths of selected PDFs in the global list pdf_paths.
    * For each selected PDF, calls the get_pdf_metadata() function to extract metadata.
    * Prints metadata (e.g., number of pages, document properties) in the console for each selected file.
* Overall: Allows users to select PDF files and view basic information about each file immediately.

In [2]:
# Block 2: Initialize PDF path list and define file selection function

# Initialize an empty list to store the paths of selected PDF files
pdf_paths = []

# Function to prompt the user to select PDF files from their system
def select_pdfs():
    global pdf_paths
    # Open a file dialog window to allow multiple PDF selection
    files = filedialog.askopenfilenames(filetypes=[("PDF Files", "*.pdf")])
    pdf_paths = list(files)  # Convert the selected paths to a list

    # Display metadata for each selected PDF using the function defined later
    for pdf_path in pdf_paths:
        metadata = get_pdf_metadata(pdf_path)  # Fetch metadata for each file
        if metadata:
            # Print out metadata details for each file
            print(f"Metadata for {pdf_path}:")
            print(f"Number of Pages: {metadata['num_pages']}")
            print("Document Info:")
            for key, value in metadata['metadata'].items():
                print(f"{key}: {value}")
            print("-" * 40)  # Separator for clarity in output

Step 3: Validate PDF Function
---
* Purpose: Checks if a selected file is a valid, readable PDF to prevent errors during extraction.
* Function: validate_pdf()
    * Attempts to open the file as a PDF using PyPDF2.
    * Returns True if the file is valid and readable, and False if not.
    * Handles errors, printing messages for both non-PDF files and other unexpected errors.
* Overall: Ensures the program processes only valid PDF files

In [3]:
# Block 3: PDF Validation Function

# This function checks if a file is a valid PDF, helping prevent errors in data extraction
def validate_pdf(file_path: str) -> bool:
    try:
        # Try to read the file as a PDF to confirm validity
        with open(file_path, 'rb') as file:
            PyPDF2.PdfReader(file)  # Attempt to initialize as a PDF
        return True  # Returns True if the file is a valid PDF
    except PyPDF2.PdfReadError:
        print(f"Error: {file_path} is not a valid PDF file.")
        return False  # Returns False if the file is invalid
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return False  # Other exceptions are also handled

Step 4: PDF Metadata Extraction Function
---
* Purpose: Extracts metadata from each PDF, helping users understand file details.
* Function: get_pdf_metadata()
    * First, calls validate_pdf() to confirm the file is a valid PDF.
    * If valid, opens the PDF and retrieves metadata, including the number of pages and any additional document information.
    * Returns a dictionary containing the page count and metadata if available.
    * Handles errors, printing messages if metadata extraction fails.
* Overall: Provides helpful file context before text extraction, such as page count and document properties

In [4]:
# Block 4: PDF Metadata Extraction Function

# This function extracts metadata from a PDF file, such as page count and document properties
def get_pdf_metadata(file_path: str) -> Optional[dict]:
    if not validate_pdf(file_path):  # Use the validate_pdf function to ensure the file is valid
        return None  # If the file is invalid, return None
    
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)  # Open the PDF reader
            # Extract metadata, including number of pages and document information
            metadata = {
                'num_pages': len(pdf_reader.pages),
                'metadata': pdf_reader.metadata  # Additional metadata
            }
            return metadata  # Return metadata as a dictionary
    except Exception as e:
        print(f"Error extracting metadata: {str(e)}")
        return None  # Return None if there is an error

Step 5: Text Extraction from PDF with Character Limit
---
* Purpose: Extracts text from each page of a PDF with an optional character limit to manage text size.
* Function: extract_text_from_pdf()
    * Checks if the file is valid by calling validate_pdf().
    * Reads the PDF page by page, extracting text.
    * Accumulates text up to a specified character limit (max_chars, default 100,000).
    * If adding a page’s text would exceed the limit, stops and adds only the remaining allowed characters.
    * Returns the extracted text up to the limit as a single string.
    * Handles errors, such as invalid or corrupted PDF files.
* Overall: Allows for controlled text extraction, providing manageable output for large PDFs.

In [5]:
# Block 5: Text Extraction from PDF with Character Limit

# This function extracts text from each PDF page, with an optional character limit
def extract_text_from_pdf(file_path: str, max_chars: int = 100000) -> Optional[str]:
    if not validate_pdf(file_path):  # Check if the file is a valid PDF
        return None  # Return None if invalid
    
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            num_pages = len(pdf_reader.pages)
            print(f"Processing PDF with {num_pages} pages...")
            
            extracted_text = []  # List to store extracted text
            total_chars = 0  # Counter for total characters
            
            # Loop through each page and extract text
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text = page.extract_text() or ""  # Get text or empty string if None
                
                # Check if adding this page's text would exceed the character limit
                if total_chars + len(text) > max_chars:
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])  # Add only the allowed characters
                    print(f"Reached {max_chars} character limit at page {page_num + 1}")
                    break  # Stop extraction if limit is reached
                
                # Append the text of this page to the list and update character count
                extracted_text.append(text)
                total_chars += len(text)
                print(f"Processed page {page_num + 1}/{num_pages}")
            
            # Join extracted text into a single string for output
            final_text = '\n'.join(extracted_text)
            print(f"\nExtraction complete! Total characters: {len(final_text)}")
            return final_text
            
    except PyPDF2.PdfReadError:
        print("Error: Invalid or corrupted PDF file")
        return None  # Handle read errors
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None  # Handle unexpected errors

Step 6: Text Extraction, Preview, and Save Function
---
* Purpose: Combines the steps of metadata display, text extraction, preview, and file saving.
* Function: extract_and_save_text()
    * Iterates through each PDF file in pdf_paths.
    * Extracts and displays metadata for each file.
    * Calls extract_text_from_pdf() for text extraction.
    * Displays the first 500 characters of each PDF’s extracted text as a preview.
    * Appends each file’s text to combined_text, ensuring the total doesn’t exceed 100,000 characters.
    * Opens a save dialog where users can select a location and name for the output file.
    * Saves combined_text to the chosen location.
* Overall: A complete extraction workflow that previews and allows saving of text output at a user-specified location.

In [6]:
# Block 6: Text Extraction, Preview, and Save Function

# Placeholder variables for save path and combined text after extraction
save_path_after_extraction = None
combined_text = ""  # Initialize empty combined text

# This function extracts text, provides a preview, and lets users save it to a specified location
def extract_and_save_text():
    global save_path_after_extraction  # Declare as global to update the variable outside this function
    global combined_text  # Declare combined_text as global to access it outside the function
    combined_text = ""  # Combined text from all PDFs
    
    # Loop through each selected PDF
    for pdf_path in pdf_paths:
        print("Extracting metadata...")
        metadata = get_pdf_metadata(pdf_path)  # Retrieve metadata
        if metadata:
            print("\nPDF Metadata:")
            print(f"Number of pages: {metadata['num_pages']}")
            print("Document info:")
            for key, value in metadata['metadata'].items():
                print(f"{key}: {value}")
        
        # Extract text for each PDF
        print("\nExtracting text...")
        extracted_text = extract_text_from_pdf(pdf_path)
        
        # Preview first 500 characters if extraction is successful
        if extracted_text:
            print("\nPreview of extracted text (first 500 characters):")
            print("-" * 50)
            print(extracted_text[:500])
            print("-" * 50)
            print(f"\nTotal characters extracted: {len(extracted_text)}")
            
            combined_text += extracted_text + "\n"  # Append to combined text
        
        # Enforce a cumulative character limit across all PDFs
        if len(combined_text) > 100000:
            combined_text = combined_text[:100000]  # Truncate if over limit
            print("Reached overall 100,000 character limit.")
            break
    
    # Prompt user to select a save location
    save_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt")])
    if save_path:
        # Save the combined text to the chosen file
        with open(save_path, "w", encoding="utf-8") as txt_file:
            txt_file.write(combined_text)
        print(f"Extracted text has been saved to {save_path}")

    # Set the global variable with the path for access outside this function
    save_path_after_extraction = save_path

Step 7: Tkinter GUI Setup
---
* Purpose: Creates a graphical interface for file selection and text extraction.
* Code Details:
    * Sets up a tkinter window titled "PDF Text Extractor."
    * Defines a button for selecting PDF files and another for extracting and saving text.
    * Runs root.mainloop() to keep the GUI active until the user exits.
* Overall: Provides an accessible GUI with buttons for PDF selection and text extraction/saving.

In [7]:
root = tk.Tk()
root.title("PDF Text Extractor")
root.geometry("400x300")

select_button = tk.Button(root, text="Select PDF Files", command=select_pdfs)
select_button.pack(pady=20)

extract_button = tk.Button(root, text="Extract Text and Save", command=extract_and_save_text)
extract_button.pack(pady=20)

root.mainloop()

Metadata for /home/mohamedashour/Documents/Projects/Notebook_LLM/Sample_pdfs/English_12_Writing Package.pdf:
Number of Pages: 44
Document Info:
/Author: Scott
/CreationDate: D:20130901150110-07'00'
/Creator: Microsoft® Word 2010
/ModDate: D:20130903123251-07'00'
/Producer: Microsoft® Word 2010
/Title: Paragraph Writing
----------------------------------------
Extracting metadata...

PDF Metadata:
Number of pages: 44
Document info:
/Author: Scott
/CreationDate: D:20130901150110-07'00'
/Creator: Microsoft® Word 2010
/ModDate: D:20130903123251-07'00'
/Producer: Microsoft® Word 2010
/Title: Paragraph Writing

Extracting text...
Processing PDF with 44 pages...
Processed page 1/44
Processed page 2/44
Processed page 3/44
Processed page 4/44
Processed page 5/44
Processed page 6/44
Processed page 7/44
Processed page 8/44
Processed page 9/44
Processed page 10/44
Processed page 11/44
Processed page 12/44
Processed page 13/44
Processed page 14/44
Processed page 15/44
Processed page 16/44
Processed

Step 8: Llama Pre-Processing
---
### Overview of the Code
This code block sets up the computing device (device) to be used for model processing, choosing between GPU and CPU based on availability. Additionally, it defines a system prompt (SYS_PROMPT) that will guide the model in performing targeted text preprocessing tasks. The prompt instructs the model on how to clean and reformat raw PDF data, providing clear guidelines on which elements to remove or modify.

### Purpose and End Result
* Purpose: To configure the device for model operations and set a detailed system prompt (SYS_PROMPT) that instructs the model on how to process and clean up raw text data. The prompt serves as a directive, outlining exactly how the model should handle text transformation.

* End Result: The code prepares the system with an appropriate device for model execution and a well-defined prompt that ensures consistent and high-quality text preprocessing across various text segments.

In [8]:
import torch  # Import torch to manage device selection for processing

# Set the processing device based on availability
# If a GPU is available, it will use "cuda"; otherwise, it defaults to "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define a system prompt that instructs the model on how to preprocess raw text
# SYS_PROMPT provides specific guidance for text cleanup and formatting adjustments
SYS_PROMPT = """
You are a world class text pre-processor, here is the raw data from a PDF, please parse and return it in a way that is crispy and usable to send to a podcast writer.

The raw data is messed up with new lines, Latex math and you will see fluff that we can remove completely. Basically take away any details that you think might be useless in a podcast author's transcript.

Remember, the podcast could be on any topic whatsoever so the issues listed above are not exhaustive.

Please be smart with what you remove and be creative ok?

Remember DO NOT START SUMMARIZING THIS, YOU ARE ONLY CLEANING UP THE TEXT AND RE-WRITING WHEN NEEDED.

Be very smart and aggressive with removing details, you will get a running portion of the text and keep returning the processed text.

PLEASE DO NOT ADD MARKDOWN FORMATTING, STOP ADDING SPECIAL CHARACTERS THAT MARKDOWN CAPITALISATION ETC LIKES.

ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok?

Here is the text:
"""

Step 9: Create Word-Bounded Chunks from Text
---
### Overview of the Code
This function, create_word_bounded_chunks, splits a large block of text into smaller chunks, ensuring that each chunk is close to a specified size (target_chunk_size) without splitting words across chunks. By maintaining word boundaries, the function produces more readable and contextually coherent chunks, which is ideal for text processing tasks that rely on complete words and phrases.

### Purpose and End Result
* Purpose: To break down a large text into smaller, word-bounded chunks based on a target chunk size. This allows for controlled, incremental processing of text data, preventing the model from handling overly large text at once and ensuring that each chunk ends at a natural word boundary.

* End Result: A list of chunks, each approximately the size of target_chunk_size, with each chunk ending on a word boundary. This prepares the text for further processing by ensuring that no words are split mid-chunk.

In [9]:
def create_word_bounded_chunks(text, target_chunk_size):
    """
    Split text into chunks at word boundaries close to the target chunk size.
    """
    
    # Split the text into individual words
    words = text.split()
    
    # Initialize a list to hold the final chunks and variables for tracking current chunk content
    chunks = []          # List to store all completed chunks
    current_chunk = []   # Temporary list to build the current chunk of words
    current_length = 0   # Counter for the length of the current chunk in characters
    
    # Iterate through each word in the text
    for word in words:
        word_length = len(word) + 1  # Calculate word length including space after it
        
        # Check if adding this word would exceed the target chunk size
        if current_length + word_length > target_chunk_size and current_chunk:
            # If the current chunk has reached its limit, join and add it to 'chunks'
            chunks.append(' '.join(current_chunk))
            
            # Reset the current chunk with the new word and update the current length
            current_chunk = [word]
            current_length = word_length
        else:
            # Add word to the current chunk and update its length
            current_chunk.append(word)
            current_length += word_length
    
    # After the loop, add the last chunk if it contains any words
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks  # Return the list of word-bounded chunks

Step 10: Load and Optimize Language Model with Accelerate
---
### Overview of the Code
This code sets up a language model using the Hugging Face Transformers library with the Accelerate library to optimize model deployment across available devices (like GPUs). Specifically, it loads Meta’s "Llama-3.1-8B-Instruct" model, a large language model intended for generating and processing natural language text, in an efficient manner suitable for larger systems or distributed setups.

### Purpose and End Result
* Purpose: The code initializes and loads a pretrained language model and tokenizer, utilizing the Accelerator class from the accelerate library to optimize performance and manage device placement automatically. This allows the model to leverage the available hardware effectively, particularly in environments with multiple GPUs or mixed-precision settings.

* End Result: A ready-to-use model and tokenizer that are optimized and prepared for high-performance inference tasks, such as generating text responses to input prompts.

In [10]:
# Importing necessary libraries for model loading and optimization
import accelerate  # Provides the Accelerator class for optimized model deployment
from accelerate import Accelerator  # Manages device placement for distributed setups
from transformers import AutoModelForCausalLM, AutoTokenizer  # For loading the model and tokenizer

# Define the model path for a default pretrained language model from Hugging Face
DEFAULT_MODEL = "meta-llama/Llama-3.1-8B-Instruct"

# Initialize the Accelerator
# Accelerator helps in handling device placement (e.g., GPU, CPU) for efficient processing
accelerator = Accelerator()

# Load the pretrained language model with specific configurations
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,              # Model identifier from Hugging Face
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory-efficient model performance
    use_safetensors=True,        # Use safe tensor format for improved loading
    device_map=device,           # Automatically map model across available devices (e.g., GPUs)
)

# Load the tokenizer associated with the language model
# Tokenizer converts text to model-compatible input and vice versa
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)

# Prepare the model and tokenizer for optimized distributed processing
# accelerator.prepare places the model and tokenizer on appropriate devices
model, tokenizer = accelerator.prepare(model, tokenizer)

/home/mohamedashour/Documents/Projects/Notebook_LLM/notebook_llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


Step 11:Process Text Chunk with Language Model
---
### Overview of the Code
This code defines a function, process_chunk, which takes a chunk of text and processes it through a language model. The function simulates a conversation by feeding the text chunk as a prompt to the model, which then generates a response based on the provided input. The function returns the model's processed output, making it useful for handling text generation in smaller, manageable segments (chunks).

### Purpose and End Result
* Purpose: To process a chunk of text using a language model in a controlled manner. Each chunk is turned into a model prompt, and the function generates a response based on it. This allows for incremental processing of large text inputs, which is particularly useful when working with memory-intensive models or managing long inputs in chunks.

* End Result: The function outputs the generated text response for each chunk, displays both input and output for monitoring purposes, and returns the processed result. The processed text is then ready for further use or to be saved.

In [11]:
def process_chunk(text_chunk, chunk_num):
    """Process a chunk of text and return both input and output for verification"""
    
    # Define a simulated conversation prompt
    # 'SYS_PROMPT' is the initial system message, and 'text_chunk' is the user-provided input
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]
    
    # Convert the conversation into a model-compatible format (prompt template)
    # 'tokenize=False' keeps the conversation in text form without immediate tokenization
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    
    # Tokenize the prompt and prepare inputs for the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate model output without computing gradients (saves memory and speeds up inference)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,         # Temperature affects randomness; lower is more focused, higher is more varied
            top_p=0.9,               # Top-p sampling controls diversity by limiting cumulative probability of choices
            max_new_tokens=512       # Limits the maximum number of tokens generated in response
        )
    
    # Decode the generated output into readable text and remove the initial prompt
    processed_text = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()
    
    # Print chunk information to monitor processing
    print(f"INPUT TEXT:\n{text_chunk[:500]}...")  # Show first 500 characters of input for inspection
    print(f"\nPROCESSED TEXT:\n{processed_text[:500]}...")  # Show first 500 characters of output
    print(f"{'='*90}\n")  # Separator for readability in console output
    
    return processed_text  # Return the processed text for further use

Step 12: Set Up Text Processing Parameters and Chunk Input Text
---
### Overview of the Code
This code block sets up the initial parameters needed for processing the input text by defining where the text data is stored (INPUT_TEXT and INPUT_TEXT_PATH) and specifying the size of each chunk (CHUNK_SIZE). It then breaks the full input text into manageable segments (chunks) based on word boundaries, making it easier to process large amounts of text piece by piece.

### Purpose and End Result
* Purpose: To prepare the text for incremental processing by dividing it into word-bounded chunks of a specified size (CHUNK_SIZE). This allows for efficient and structured processing of large texts, helping to avoid memory limitations and enabling the model to process text in portions.

* End Result: The code produces a list of chunks, each containing a portion of the input text limited to CHUNK_SIZE. The variable num_chunks holds the total number of chunks created, which provides insight into the workload for further processing.

In [12]:
# Assign the combined extracted text to INPUT_TEXT for processing
INPUT_TEXT = combined_text  # Full text input to be processed in chunks

# Store the path where the extracted text is saved
INPUT_TEXT_PATH = save_path_after_extraction  # Location of the original extracted text file

# Define the chunk size
CHUNK_SIZE = 1000  # Number of characters per chunk, adjust based on memory and processing needs

# Break the full input text into smaller chunks, each limited by CHUNK_SIZE
# create_word_bounded_chunks ensures that chunks do not break words in half
chunks = create_word_bounded_chunks(INPUT_TEXT, CHUNK_SIZE)

# Calculate the total number of chunks created
num_chunks = len(chunks)  # Useful for tracking and monitoring progress


In [13]:
#This step is all about printing the number of processed text chunks
num_chunks

98

Step 13: Load Input Text from File and Set Up Output File Path
---
### Overview of the Code
This code block reads the input text from a file (INPUT_TEXT_PATH) and calculates the number of chunks based on a predefined chunk size (CHUNK_SIZE). It also generates a name and path for the output file, ensuring it will be saved in the same directory as the input file. This setup is essential for organizing and storing the processed text output in an orderly manner.

### Purpose and End Result
* Purpose: To load the input text from the specified file path, calculate the number of chunks required for processing, and define a path for saving the processed output. By creating the output file in the same directory as the input file, it keeps the original and processed files together, which helps with organization and access.

* End Result: The input text is loaded and ready for chunked processing, the number of chunks is determined, and a designated output file path (output_file_path) is set for saving the final processed text.

In [14]:
import os  # Import the os library for file and path management

# Read the input text file specified by INPUT_TEXT_PATH
# Opens the file in read mode and reads its entire content into the variable 'text'
with open(INPUT_TEXT_PATH, 'r', encoding='utf-8') as file:
    text = file.read()  # Store the full content of the file for further processing

# Calculate the number of chunks required to process the entire text
# The formula divides the length of the text by CHUNK_SIZE and rounds up if needed
num_chunks = (len(text) + CHUNK_SIZE - 1) // CHUNK_SIZE

# Set up the output file name and path for saving processed text
# Generate a new file name by prefixing "clean_" to the original file name from INPUT_TEXT_PATH
output_file_name = f"clean_{os.path.basename(INPUT_TEXT_PATH)}"

# Create the output file path by joining the input file's directory with the new output file name
output_file_path = os.path.join(os.path.dirname(INPUT_TEXT_PATH), output_file_name)

Step 15: Process Text Chunks and Save Processed Output
---
### Overview of the Code
This code block processes each chunk of text individually, appending the results to processed_text and writing each processed chunk directly to an output file. It utilizes the tqdm library to display a progress bar, giving visual feedback on the chunk processing progress. By saving each chunk immediately to the output file, this approach avoids potential memory issues when handling large text data.

### Purpose and End Result
* Purpose: To process each chunk of the input text incrementally, accumulate the processed results, and save each processed chunk to an output file in real time. This method allows for efficient processing and storage of large files, as each chunk is handled and saved independently.

* End Result: The full processed text is saved to an output file (output_file_path), with processed_text containing all processed chunks in memory for further use or inspection.

In [15]:
from tqdm import tqdm  # Import tqdm to display a progress bar for the loop

# Initialize a string to accumulate all processed text chunks
processed_text = ""

# Open the output file in write mode
# 'output_file_path' is the file path where the processed text will be saved
with open(output_file_path, 'w', encoding='utf-8') as out_file:
    # Loop through each chunk in 'chunks', using tqdm to show progress
    for chunk_num, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):
        
        # Process the current chunk with the process_chunk function
        processed_chunk = process_chunk(chunk, chunk_num)
        
        # Append the processed chunk to the overall processed_text variable
        processed_text += processed_chunk + "\n"
        
        # Write the processed chunk immediately to the output file
        out_file.write(processed_chunk + "\n")
        
        # Flush the file buffer to ensure the chunk is written to disk
        out_file.flush()

# Print confirmation of where the processed text has been saved
print(f"Processed text has been saved to: {output_file_path}")

Processing chunks:   1%|          | 1/98 [00:02<03:53,  2.40s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
Scott Findley School District 43 Gleneagle Secondary Paragraph Writing pg. 2 TAG TIQs CE & Get to the Point pg. 10 Get To The Point pg. 11 State/Quote/Clarify pg. 12 Paragraph Response Writing pg. 16 Foundational Writing pg. 19 Essay Writing Essential Guide pg. 21 Writing Terms 12 pg. 27 Academic Writing pg. 34 Writing a Timed Essay pg. 35 Using Quotations pg. 36 Key Verbs in Written Response Questions pg. 38 Tone & Formality in Academic Writing pg. 39 New Rules for Editing pg 41 Essay Checklist...

PROCESSED TEXT:
uccinctly express oneself in a written format. No one is naturally gifted or born a writer; it is a skill learned through process and practice. The more you properly exercise your writing abilities, the stronger they will become. To improve your writing, start by being a good reader....



Processing chunks:   2%|▏         | 2/98 [00:06<05:23,  3.37s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
well written work of others in novels, stories, and articles will help you learn new ways to express your own ideas.  Plan before you write anything important – sometimes just 5 minutes will help you plan and organiz e your ideas and result in stronger results.  Complete your written assignments early so that you can put them aside to come back the next day to correct a nd proof them. When writing anything, make sure that you stress that the idea/topic/theme is important to the reader/world: e...

PROCESSED TEXT:
ill help you plan and organize your ideas and result in stronger results. 
Complete your written assignments early so that you can put them aside to come back the next day to correct and proof them. 
When writing anything, make sure that you stress that the idea/topic/theme is important to the reader/world: essentially, answer the question 'why do we care about what you are writing?' 
A title is a crucial part of your writing that attracts and piques the reader's

Processing chunks:   3%|▎         | 3/98 [00:10<05:42,  3.61s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

INPUT TEXT:
that you’re hungry and looking for a friend to go to lunch with. How would you ask that person to join you? Now imagine that you’re asking your grandmother the same question. What about your boss? A client? Your professor? The president? You would probably choose different phrasing for each of these individuals. Why is that? When you sit down to write, consider whether your audience falls into one of these categories (or others you might have thought of). Try to choose language that seems approp...

PROCESSED TEXT:
se. For instance, you might ask a friend to lunch with "Hey, want to grab lunch together?" while your grandmother might appreciate "Grandma, would you like to join me for lunch?" Your boss might be asked "May I invite you to lunch?" and a client could be asked "Would you like to join me for a working lunch?" The president would likely require a more formal invitation. 

When writing for an academic audience, use a formal tone by avoiding personal pronouns like "y

Processing chunks:  99%|█████████▉| 97/98 [04:38<00:03,  3.61s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
conversational for academic writing. 39 Slang is appropriate for conversations and e-mails, but not research papers, or other formal writing. If you’re not sure if it’s slang, pick another word or phrase just to be on the safe side. 3. Avoid run -on sentences. Run-on sentences are common in speech and other informal interactions because we just keep adding on to something we wanted to say. In writing, however, these run -ons become distracting and hard to follow. 4. Leave the IM spellings and ca...

PROCESSED TEXT:
ing requires more polished language. Slang is not recommended for research papers, other formal writing, or academic writing. If unsure, use an alternative phrase to be safe. 

Avoid run-on sentences in formal writing. Run-on sentences can be distracting and hard to follow. Instead, use clear and concise sentences. 

Standard spelling, punctuation, and capitalization should be used in academic writing, even in instant messages. In formal writing, use proper langu

Processing chunks: 100%|██████████| 98/98 [04:40<00:00,  2.86s/it]

INPUT TEXT:
subjects. 2. Prepositions are not words to ends sentences with. Do not end sentences with prepostions. 3. And don’t start a sentence with a conjuncti on. Don’t start a sentence with a conjuction. 4. It is wrong to ever split an infiniti ve It is wrong to split an infinitive. 5. Avoid cliches like the plague. (They’re old ha t) Avoid using clichés in your writing. 6. Also, always avoid annoying alliter ation. Avoid using alliteration. 7. Be more or less specific. Be specific in your writing. 8. P...

PROCESSED TEXT:
conjunction. It's wrong to split an infinitive. Avoid using clichés. Avoid using alliteration. Be specific in your writing. Parenthetical remarks are unnecessary. Never use repetitive redundancies....

Processed text has been saved to: /home/mohamedashour/Documents/Projects/Notebook_LLM/Sample_pdfs/clean_extracted_text.txt


Step 16: Display Processing Summary and Preview Processed Text
---
### Overview of the Code
This code block provides a summary of the text processing, displaying key information such as the input and output file paths and the total number of chunks processed. It also includes a preview of the processed text, showing both the beginning and the end of the processed_text content. This allows for a quick inspection of the final output, ensuring that the text has been processed as expected.

### Purpose and End Result
* Purpose: To provide an overview of the processing results, including important file paths and the number of chunks processed, and to display a preview of the final processed text. This summary serves as a checkpoint, allowing users to verify that the input was correctly processed and saved.

* End Result: A summary of the processing operation, including a preview of the processed text. The beginning and end sections of processed_text are displayed, offering insight into the transformation and completeness of the output.

In [16]:
# Print a confirmation that the entire processing operation is complete
print(f"\nProcessing complete!")

# Display key information for reference
print(f"Input file: {INPUT_TEXT}")  # Path or content of the input text
print(f"Output file: {output_file_name}")  # Path of the output file where processed text is saved
print(f"Total chunks processed: {num_chunks}")  # Total number of chunks processed

# Preview the beginning and end of the complete processed text
# This provides a quick insight into the quality and content of the processed output
print("\nPreview of final processed text:")

# Display the beginning of the processed text (first 1000 characters)
print("\nBEGINNING:")
print(processed_text[:1000])  # Shows first 1000 characters of processed text

# Display the end of the processed text (last 1000 characters)
print("\n...\n\nEND:")
print(processed_text[-1000:])  # Shows last 1000 characters of processed text


Processing complete!
Input file:  
Scott Findley  
School District 43  
Gleneagle Secondary  
Paragraph Writing   
pg. 2 
TAG TIQs CE  & Get to 
the Point  
pg. 10  
Get To The Point  
pg. 11  
State/Quote/Clarify  
pg. 12 
Paragraph Response 
Writing  
pg. 16  
Foundational Writing  
pg. 19 
Essay Writing Essential 
Guide  
pg. 21 
Writing Terms 12  
pg. 27  
Academic Writing  
pg. 34 
Writing a Timed Essay  
pg. 35  
Using Quotations  
pg. 36 
Key Verbs in Written 
Response Questions  
pg. 38 
Tone & Formality in 
Academic Writing  
pg. 39 
New Rules for Editing  
pg 41 
Essay Checklists  
pg. 42  
 
 E12 WRITING  
At the heart of the English curriculum is being able to clearly and 
succinctly express oneself in a written format. Contrary to belief, 
no one is a ‘naturally gifted’ or ‘born writer’ ; it is a skill that is 
learned through process and practice. Like a muscle, the more you 
properly  exercise your writing abilities, the stronger they will 
become.   
 
So if you need t